In [1]:
import pandas as pd
import datetime
import warnings
warnings.filterwarnings("ignore")  # 忽略UserWarning兼容性警告

yesterday = (datetime.date.today() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
now = datetime.date.today().strftime("%Y-%m-%d")
print(now,yesterday)

Discount_yesterday = float('0.80')
Discount_now       = float('0.80')
df_jp              = pd.read_excel(f'C:/Users/15516/Desktop/数据处理/竞对数据/比对目录——2023.07.24版.xlsx',sheet_name='比价明细')
df_YE_yesterday   = pd.read_csv(f'C:/Users/15516/Desktop/数据处理/竞对数据/牙E在线/牙E在线{yesterday}.csv',encoding='GBK')
df_YE_now         = pd.read_csv(f'C:/Users/15516/Desktop/数据处理/竞对数据/牙E在线/牙E在线{now}.csv',encoding='GBK')

2023-07-28 2023-07-27


In [2]:
def deal_with(df, Discount):
    """
    对DataFrame进行处理，添加折扣价格列，并返回处理后的DataFrame。

    参数：
    df (pandas.DataFrame): 要处理的DataFrame。
    Discount (float): 折扣值。

    返回：
    pandas.DataFrame: 处理后的DataFrame。
    """

    # 去除型号列中的空格
    df['型号'] = df['型号'].replace(' ', '', regex=True)

    # 拼接链接、型号和商品编号列为品型列
    df['品型'] = df['链接'].astype(str) + df['型号'].astype(str) + df['商品编号'].astype(str)

    # 重新选择列，只保留品型、价格和库存列
    df = df[['品型', '价格', '库存']]

    # 计算折扣价格列
    df['折扣价格'] = df['价格'] * Discount

    return df

def zhuijia(file,key):
    
    # 遍历除了产品编码列之外的所有列
    for col in file.columns:
        if col != '品型':
            file.rename(columns={col: col + f'-{key}'}, inplace=True)
            
    return file

In [3]:
# 处理昨天的文件
file_yesterday = deal_with(df_YE_yesterday, Discount_yesterday)

# 处理当前的文件
file_now = deal_with(df_YE_now, Discount_now)

# 合并品型列，去重，并重置索引
df_combined = pd.concat([file_now['品型'], file_yesterday['品型']]).drop_duplicates().reset_index(drop=True)

# 创建合并后的DataFrame
df_merged = pd.DataFrame({'品型': df_combined})

# 添加备注，表示昨天的数据
file_yesterday = zhuijia(file_yesterday, "前")

# 添加备注，表示当前的数据
file_now = zhuijia(file_now, "后")

# 根据品型列左连接合并昨天和当前的数据
df_merged = df_merged.merge(file_yesterday, how='left', on='品型').merge(file_now, how='left', on='品型')

# 根据条件生成上下架状态列
df_merged['上下架状态'] = df_merged.apply(lambda row: '新上架' if pd.isnull(row['价格-前']) else ('下架' if pd.isnull(row['价格-后']) else '上架'), axis=1)

# 根据条件生成折扣价格变动列
df_merged['折扣价格变动'] = df_merged.apply(lambda row: '涨价' if row['折扣价格-后'] > row['折扣价格-前'] else ('不变' if row['折扣价格-后'] == row['折扣价格-前'] else ('降价' if row['折扣价格-后'] < row['折扣价格-前'] else '不变')), axis=1)

# 根据条件生成库存状态列
df_merged['库存状态'] = df_merged.apply(lambda row: '上升' if row['库存-后'] > row['库存-前'] else ('不变' if row['库存-后'] == row['库存-前'] else ('下降' if row['库存-后'] < row['库存-前'] else '不变')), axis=1)

# 根据条件生成库存变动量列
df_merged['库存变动量'] = df_merged.apply(lambda row: row['库存-后'] - row['库存-前'] if row['库存状态'] != '不变' else '', axis=1)

# 根据条件生成折扣价格变动量列
df_merged['折扣价格变动量'] = df_merged.apply(lambda row: row['折扣价格-后'] - row['折扣价格-前'] if row['折扣价格变动'] != '不变' else '', axis=1)

# 打印整理后的DataFrame
df_merged = df_merged.merge(df_jp, how='left', left_on='品型', right_on='牙E在线品型')

df_merged = df_merged[['品型','上下架状态','折扣价格变动','库存状态','价格-前','价格-后','折扣价格-前','折扣价格-后','折扣价格变动量','库存-后','库存变动量','励齿编码']]

df_merged.to_excel(f'C:/Users/15516/Desktop/牙E变化{now}.xlsx', index=False)

In [4]:
df_merged

,品型,上下架状态,折扣价格变动,库存状态,价格-前,价格-后,折扣价格-前,折扣价格-后,折扣价格变动量,库存-后,库存变动量,励齿编码
0,雅克菱 天然蛋白酶美白牙粉50g抹茶味（粉色）3260009002,上架,不变,不变,18.0,18.0,14.4,14.4,,30.0,,NaN
1,雅克菱 天然蛋白酶美白牙粉50g薄荷味（绿色）3260009002,上架,不变,下降,18.0,18.0,14.4,14.4,,34.0,-1.0,NaN
2,严迪 皓齿便携式冲牙器喷头补充装（新包装）3*1FC1593270052001,上架,不变,不变,15.5,15.5,12.4,12.4,,141.0,,NaN
3,GUM Y型牙线棒/签30*130枚200016001,上架,不变,不变,38.0,38.0,30.4,30.4,,1036.0,,NaN
4,诊所定制 护理牙线棒（5.4元/盒 1000盒起订）50*12000盒/箱780042002,上架,不变,不变,16000.0,16000.0,12800.0,12800.0,,500.0,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13963,康田正 口腔材料注射头100*1I0T04（小尖尖）1690257001,下架,不变,不变,35.0,NaN,28.0,NaN,,NaN,,NaN
13964,普特 动物牵引橡皮圈50*1（整盒）企鹅5/16（3.5OZ）3000069001,下架,不变,不变,175.0,NaN,140.0,NaN,,NaN,,NaN
13965,普特 动物牵引橡皮圈50*1（整盒）猴子3/8（3.5OZ）3000068005,下架,不变,不变,175.0,NaN,140.0,NaN,,NaN,,NaN
13966,普特 结扎圈/橡皮圈50*14019-9（银灰色）3000066002,下架,不变,不变,45.0,NaN,36.0,NaN,,NaN,,NaN
